In [ ]:
import pandas as pd
import numpy as np


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np

In [5]:
data = pd.read_csv('/content/drive/MyDrive/twitter-spam/train.csv')
data.head()

,Id,Tweet,following,followers,actions,is_retweet,location,Type
0,10091,It's the everything else that's complicated. #...,0.0,11500.0,NaN,0.0,Chicago,Quality
1,10172,Eren sent a glare towards Mikasa then nodded a...,0.0,0.0,NaN,0.0,NaN,Quality
2,7012,I posted a new photo to Facebook http://fb.me/...,0.0,0.0,NaN,0.0,"Scotland, U.K",Quality
3,3697,#jan Idiot Chelsea Handler Diagnoses Trump Wit...,3319.0,611.0,294.0,0.0,"Atlanta, Ga",Spam
4,10740,Pedophile Anthony Weiner is TERRIFIED of Getti...,4840.0,1724.0,1522.0,0.0,Blumberg,Spam


In [6]:
data = data.drop(['Id','following', 'followers', 'actions', 'is_retweet', 'location'], axis=1)

data.rename(columns={'Type': 'Class', 'Tweet': 'Message'}, inplace=True)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11968 entries, 0 to 11967
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Message  11968 non-null  object
 1   Class    11968 non-null  object
dtypes: object(2)
memory usage: 187.1+ KB


In [7]:
data['Class'].value_counts()


Quality    6153
Spam       5815
Name: Class, dtype: int64

In [9]:
import re
!pip install contractions
import contractions
import nltk
nltk.download('stopwords')
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

stopwords = nltk.corpus.stopwords.words('english')

wnl = WordNetLemmatizer()

def lemmatizer(tokens):
    '''This function takes a preprocessed tokenized text, lemmatizes the word and returns a
    joint string.'''
    
    #Tag tokens in order to obtain the grammatical category (it is needed in the lemmatizer)
    tagged_tokens = nltk.pos_tag(tokens, tagset='universal')
    
    #Create a dictionary containing the gr
    tag_map = {'j': wordnet.ADJ, 'v': wordnet.VERB, 'n': wordnet.NOUN, 'r': wordnet.ADV}
    
    #App
    new_tagged_tokens = [(word, tag_map.get(tag[0].lower(), wordnet.NOUN)) 
                         for word, tag in tagged_tokens]
    
    #Lemmatize
    lemmatized_text = ' '.join(wnl.lemmatize(word, tag) for word, tag in new_tagged_tokens)
    
    return lemmatized_text

def text_parser(text):
    '''This function takes a raw string and removes special characters, converts it to lowercase,
    expands the contractions, removes additional ' symbols, tokenize, removes stopwords and lemmatizes
    '''
    
    #Use regex to filter non desired characters (note that £, $, € are important as spam messages
    #normally contain these symbols related to money)
    text = re.sub(r"[^A-Za-z0-9£$'€\s]", '', str(text))
    
    text = text.lower()
    
    #Expand contracted expressions
    text = contractions.fix(text)
    
    #Filter again, this time not including ' (it wasn't filtered before because it was left for the contractions)
    text = re.sub(r"[^A-Za-z0-9£$€\s]", '', text)
    
    #Tokenize text
    text = nltk.word_tokenize(text)
    
    #Remove stopwords from the tokenize text
    text = [word for word in text if word not in stopwords]
    
    #Lemmatize the text (it uses the previously defined function)
    
    text = lemmatizer(text)
    
    return text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [19]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')  
nltk.download('universal_tagset')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [21]:
data['Message'] = data['Message'].apply(text_parser)

data.head()

,Message,Class
0,everything else complicate pesummit pxpictwitt...,Quality
1,eren send glare towards mikasa nod stand go he...,Quality
2,post new photo facebook httpfbme2be7liyuj,Quality
3,jan idiot chelsea handler diagnose trump disea...,Spam
4,pedophile anthony weiner terrify get beat pris...,Spam


In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

#Convert the Message column to a numpy array
words = np.array([a for a in data['Message']])

#create TF-IDF vectorizer
tf = TfidfVectorizer(min_df=0., max_df=1., use_idf=True)

#Fit the data, create a matrix and a new dataframe with the transformed values
tf_mat = tf.fit_transform(words)

tf_mat = tf_mat.toarray()

new_data = pd.DataFrame(tf_mat)

new_data = pd.concat((new_data, data['Class']), axis=1)

new_data

,0,1,2,3,4,5,6,7,8,9,...,29703,29704,29705,29706,29707,29708,29709,29710,29711,Class
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Quality
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Quality
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Quality
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Spam
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Spam
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11963,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Quality
11964,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Quality
11965,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Spam
11966,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Quality


In [23]:
import spacy

!python -m spacy download en_core_web_md

#Load spacy pretrained model
model = spacy.load('en_core_web_md')

#Apply model to the parsed text from the dataset
embeds = data['Message'].apply(model)

#Create a matrix containing the embedded text
matrix = np.array([v.vector for v in embeds])

#Create a dataframe containig the final vectors
vectorized_text = pd.DataFrame(matrix)

vectorized_text

2022-09-18 18:27:32.904074: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42.8 MB 2.4 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,1.094952,0.153610,-1.180824,-0.543990,0.328840,0.624904,1.379760,2.150680,-1.066680,1.639244,...,-0.456734,0.663450,1.599470,-0.673836,-1.462056,-0.080408,1.056196,1.218186,-2.183780,0.851252
1,-1.258133,1.674885,-2.204869,0.513586,-0.144375,0.002368,0.915517,0.341146,-2.437486,1.631344,...,-1.107361,-0.503521,0.034087,-0.805186,-0.231276,0.484844,-2.502356,1.360384,-1.648665,0.776031
2,2.238620,-0.474886,-0.679706,-0.387828,2.038980,-1.395024,3.463180,2.021508,-2.953560,1.370847,...,2.988360,0.924306,0.486054,1.226828,-0.709820,-0.206240,1.760380,-1.549030,-3.692820,-1.264237
3,1.017248,0.558851,-1.113626,0.635406,1.349468,-0.287413,-1.581300,2.786480,-0.820020,0.845358,...,1.391965,-1.333850,1.507993,-0.072141,-0.351645,0.407744,1.393300,0.081281,0.644076,0.969711
4,-1.567257,0.031845,-0.613045,0.754732,0.309937,0.589292,1.640558,3.099050,-0.292940,-0.036445,...,1.387069,-1.614989,0.834921,-0.429525,-0.477646,-0.875268,-0.882536,1.162460,0.038680,-0.548650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11963,-2.930737,-0.334167,0.649567,-2.521463,-0.838923,0.133797,1.239767,3.545500,-0.537697,1.619467,...,0.488040,-0.807913,-0.806220,0.403893,-0.484487,-1.741900,-1.461761,3.380033,0.654760,2.971600
11964,1.062251,-1.072090,-2.717472,-1.679454,1.029217,-0.761321,1.446629,2.974726,-0.079420,-1.164438,...,1.506229,-0.731936,1.713928,-0.990300,-1.316903,-2.385814,1.890359,-1.243643,-1.725877,0.581799
11965,-1.059920,0.330291,-0.396427,-0.371471,2.434743,1.122429,2.042557,2.199976,-1.017850,0.804674,...,0.197556,-0.635019,0.641889,1.687455,-0.842514,0.980286,0.963823,-0.150047,-1.673164,-0.629170
11966,0.526322,-0.727608,-0.165302,0.693584,1.613972,-1.151016,1.291410,0.667534,-1.087508,-0.353154,...,0.745895,0.758057,0.222625,-0.184368,0.717808,0.890088,1.411823,-0.481272,-1.492049,1.035575


In [24]:
y = pd.get_dummies(data['Class'], drop_first=True)

In [25]:
from sklearn.model_selection import train_test_split, GridSearchCV

X = vectorized_text

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify=y, random_state=0)

In [26]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=0)

rf.fit(X_train, y_train)

pred = rf.predict(X_test)

accuracy = rf.score(X_test, y_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


In [27]:
print(accuracy)

0.7635756056808688


In [32]:
import pickle

In [33]:
pickle.dump(rf, open('model.pkl', 'wb'))

In [34]:
pickled_model = pickle.load(open('model.pkl', 'rb'))
pickled_model.predict(X_test)

array([1, 0, 0, ..., 0, 0, 1], dtype=uint8)